In [1]:
import numpy as np
import networkx as nx
import string

f=open('input.txt') #not with read because thats probably the whole file
lines = [list(line.rstrip('\n')) for line in f]
grid = np.array(lines)

def getneigh(i): # to determine neighboring cells
  res = []
  if (i[0],i[1]+1) in locations: res.append((i,(i[0],i[1]+1)))
  if (i[0],i[1]-1) in locations: res.append((i,(i[0],i[1]-1)))
  if (i[0]+1,i[1]) in locations: res.append((i,(i[0]+1,i[1])))
  if (i[0]-1,i[1]) in locations: res.append((i,(i[0]-1,i[1])))
  return res

def innerrouter(input):
    if input[0]<2 or input[0]>grid.shape[0]-3 or input[1]<2 or input[1]>grid.shape[1]-3:
        return -1
    else: return 1

def getneighportals(i):
  if (i[0],i[1]+1) in portals.keys(): return {grid[(i[0],i[1]+1)],grid[(i[0],i[1]+2)]},innerrouter((i[0],i[1]+1))
  if (i[0],i[1]-1) in portals.keys(): return {grid[(i[0],i[1]-1)],grid[(i[0],i[1]-2)]},innerrouter((i[0],i[1]-1))
  if (i[0]+1,i[1]) in portals.keys(): return {grid[(i[0]+1,i[1])],grid[(i[0]+2,i[1])]},innerrouter((i[0]+1,i[1]))
  if (i[0]-1,i[1]) in portals.keys(): return {grid[(i[0]-1,i[1])],grid[(i[0]-2,i[1])]},innerrouter((i[0]-1,i[1]))

locations = {tuple(i)  for i in np.argwhere(grid=='.')}
portals = np.argwhere(np.isin(grid, list(string.ascii_uppercase)))
portals = {tuple(p): grid[tuple(p)] for p in portals}
adj_to_portal = {l: getneighportals(l) for l in locations if getneighportals(l)}

for k,v in adj_to_portal.items():
    if len(v[0])==1:
        if 'A' in v[0]: start = k
        if 'Z' in v[0]: end = k

In [2]:
g = nx.Graph()
g.add_nodes_from(locations)
for a in locations: # for each location (without the doors)
    g.add_edges_from(getneigh(a))

count = 0
for k,v in adj_to_portal.items():
    if v[0]!={'A'} and v[0]!={'Z'}:
        for l,w in adj_to_portal.items():
            if v[0]==w[0] and k!=l:
                # print(k,l)
                count+=1
                g.add_edge(k,l)
nx.shortest_path_length(g,start,end)

522

In [3]:
# part2
def getneigh(inp):
  res = []
  i = inp[0]
  if (i[0],i[1]+1) in locations: res.append((inp,((i[0],i[1]+1),inp[1])))
  if (i[0],i[1]-1) in locations: res.append((inp,((i[0],i[1]-1),inp[1])))
  if (i[0]+1,i[1]) in locations: res.append((inp,((i[0]+1,i[1]),inp[1])))
  if (i[0]-1,i[1]) in locations: res.append((inp,((i[0]-1,i[1]),inp[1])))
  return res

def getneighportals(inp,layer):
    i = inp[0]
    if (i[0],i[1]+1) in portals.keys(): return {grid[(i[0],i[1]+1)],grid[(i[0],i[1]+2)]},layer+innerrouter((i[0],i[1]+1))
    if (i[0],i[1]-1) in portals.keys(): return {grid[(i[0],i[1]-1)],grid[(i[0],i[1]-2)]},layer+innerrouter((i[0],i[1]-1))
    if (i[0]+1,i[1]) in portals.keys(): return {grid[(i[0]+1,i[1])],grid[(i[0]+2,i[1])]},layer+innerrouter((i[0]+1,i[1]))
    if (i[0]-1,i[1]) in portals.keys(): return {grid[(i[0]-1,i[1])],grid[(i[0]-2,i[1])]},layer+innerrouter((i[0]-1,i[1]))

layerlocations = [(x[0],0) for x in zip(locations)]
adj_to_portal = {l: getneighportals(l,0) for l in layerlocations if getneighportals(l,0)}
for k,v in adj_to_portal.items():
    if len(v[0])==1:
        if 'A' in v[0]: start = k
        if 'Z' in v[0]: end = k
        
g = nx.DiGraph()
numlayers = 30
for layer in range(numlayers):
    layerlocations = [(x[0],layer) for x in zip(locations)]
    # print(layerlocations)
    adj_to_portal = {l: getneighportals(l,layer) for l in layerlocations if getneighportals(l,layer)}

    g.add_nodes_from(layerlocations)
    for a in layerlocations: # for each location (without the doors)
        g.add_edges_from(getneigh(a))

    count = 0
    for k,v in adj_to_portal.items(): # k = (loc, layer), v = connected portal numbers, layer)
        if v[0]!={'A'} and v[0]!={'Z'}:
            if numlayers>v[1]>=0:
                for l,w in adj_to_portal.items():
                    if numlayers>l[1]>=0:
                        if v[0]==w[0] and k!=l:
                            count+=1
                            g.add_edge(k,(l[0],v[1]))
nx.shortest_path_length(g,start,end)

6300